In [1]:
import ee
import geemap
import sys,os
from GEE_Func.S1_distor_dedicated import load_S1collection,S1_CalDistor
from GEE_Func.S2_filter import merge_s2_collection
from GEE_Func.GEE_DataIOTrans import BandTrans,DataTrans,DataIO
from GEE_Func.GEE_CorreterAndFilters import S1Corrector
from GEE_Func.GEEMath import get_minmax
import math
from functools import partial
from tqdm import tqdm 

# 重载函数
# import importlib
# importlib.reload(S1_distor_dedicated)

# geemap.set_proxy(port=10809)
ee.Authenticate()
ee.Initialize()

## 基础数据导入

In [2]:
# 获取S1 、 DEM
AOI =  ee.Geometry.Polygon([[[91.18640929255166, 29.43865075199293],
   [91.2501966942778, 29.43865075199293],
   [91.2501966942778, 29.498705908528617],
   [91.18640929255166, 29.498705908528617],
   [91.18640929255166, 29.43865075199293]]])

def split_rectangle_into_grid(AOI, rows, cols):
    # 获取AOI的边界坐标
    coords = ee.List(AOI.coordinates().get(0))
    x_min = ee.Number(ee.List(coords.get(0)).get(0))
    y_min = ee.Number(ee.List(coords.get(0)).get(1))
    x_max = ee.Number(ee.List(coords.get(2)).get(0))
    y_max = ee.Number(ee.List(coords.get(2)).get(1))
    
    # 计算每个小矩形的宽度和高度
    width = x_max.subtract(x_min).divide(cols)
    height = y_max.subtract(y_min).divide(rows)
    
    # 定义一个函数来创建小矩形
    def create_rectangle(row, col):
        x1 = x_min.add(col.multiply(width))
        y1 = y_min.add(row.multiply(height))
        x2 = x1.add(width)
        y2 = y1.add(height)
        return ee.Geometry.Rectangle([x1, y1, x2, y2])
    
    # 生成小矩形列表
    grid_list = []
    for row in range(rows):
        for col in range(cols):
            rectangle = create_rectangle(ee.Number(row), ee.Number(col))
            grid_list.append(rectangle)
    
    return ee.List(grid_list)

AOIs = split_rectangle_into_grid(AOI,3,3)
AOI = AOIs.get(0)

year = '2019'
START_DATE  = ee.Date(year + '-01-01')
END_DATE   = ee.Date(year + '-12-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')
Origin_scale = 10
Prj_scale = 30


s1_ascending, s1_descending = load_S1collection(AOI,START_DATE,END_DATE,MIDDLE_DATE,FilterSize=30)
DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()

S1_image = s1_ascending
DEM = DEMSRTM
Projection = S1_image.select(0).projection()
Mask = S1_image.select(0).mask()

# 获取辅助线
azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict  = S1Corrector.getS1Corners(s1_ascending, AOI,'ASCENDING')

Heading = azimuthEdge.get('azimuth')
s1_azimuth_across = ee.Number(Heading).subtract(90.0) # 距离向
Auxiliarylines = ee.Geometry.LineString([startpoint, endpoint])

Eq_pixels = DataTrans.Eq_pixels

BandTrans.delBands(S1_image, ['VH', 'VV'])
Cal_image = (Eq_pixels(BandTrans.delBands(S1_image, 'VV','VH').resample('bicubic')).rename('angle')
                      .addBands(ee.Image.constant(0).rename('Leftlayover'))
                      .addBands(ee.Image.constant(0).rename('Rightlayover'))
                      .addBands(ee.Image.constant(0).rename('Shadow'))
                      .addBands(ee.Image.pixelCoordinates(Projection))
                      .addBands(DEM.select('elevation'))
                      .addBands(ee.Image.pixelLonLat())
                      .updateMask(Mask)
                      .reproject(crs=Projection, scale=Prj_scale)
                      .clip(AOI))
Templist_A = S1_CalDistor.AuxiliaryLine2Point(s1_azimuth_across, coordinates_dict, Auxiliarylines,AOI, Prj_scale)

Without Filter


In [4]:
Cal_image

## 面转线并采用Aoi取交集过滤-->获取4、9邻域-->计算代求点高程、x\y、入射角
（过滤点数不满足要求的ee.List以及点邻域取值不够的数据点）

In [5]:
import numpy as np
from scipy.optimize import least_squares
from tqdm import tqdm

def Line2Points(feature,region,scale=30):
    # 从Feature中提取线几何对象
    line_geometry = ee.Feature(feature).geometry().intersection(region)
    
    # 获取线的所有坐标点
    coordinates = line_geometry.coordinates()
    start_point = ee.List(coordinates.get(0)) # 起点坐标
    end_point = ee.List(coordinates.get(-1))  # 终点坐标
    
    # 获取线段的总长度
    length = line_geometry.length()

    # 计算插入点的数量
    num_points = length.divide(scale).subtract(1).floor()

    # 计算每个间隔点的坐标
    def interpolate(i):
        i = ee.Number(i)
        fraction = i.divide(num_points)
        interpolated_lon = ee.Number(start_point.get(0)).add(
            ee.Number(end_point.get(0)).subtract(ee.Number(start_point.get(0))).multiply(fraction))
        interpolated_lat = ee.Number(start_point.get(1)).add(
            ee.Number(end_point.get(1)).subtract(ee.Number(start_point.get(1))).multiply(fraction))
        return ee.Feature(ee.Geometry.Point([interpolated_lon, interpolated_lat]))

    # 使用条件表达式过滤长度为0的线段
    filtered_points = ee.FeatureCollection(ee.Algorithms.If(num_points.gt(0), 
                                                            ee.FeatureCollection(ee.List.sequence(1, num_points).map(interpolate)),
                                                            ee.FeatureCollection([])))

    return filtered_points

def filter_Listlen(item,len=3):
    '''将list长度小于3的过滤，该函数len=3起步判断，由于len=2的时候会将单点判定为正确，故存在误差为了节省计算量，未对此处进行修改'''
    item_list = ee.List(item)
    return ee.Algorithms.If(item_list.size().gte(len), item_list, None)

def get_neighborhood_info(point,image,scale=30,neighborhood_type='4',Pointfilter=True):
    
    point = ee.Geometry.Point(point)
    buffer_region = point.buffer(30)

    # 生成矩形区域
    region = buffer_region.bounds()
    # 获取矩形的坐标
    coords  = ee.List(region.coordinates().get(0))

    # 提取四个角点坐标
    corner1 = ee.Geometry.Point(ee.List(coords.get(0)))
    corner2 = ee.Geometry.Point(ee.List(coords.get(1)))
    corner3 = ee.Geometry.Point(ee.List(coords.get(2)))
    corner4 = ee.Geometry.Point(ee.List(coords.get(3)))
    
    if neighborhood_type=='4':
        # 创建包含所有点的MultiPoint几何对象
        region_coord = ee.Geometry.MultiPoint([
                                            corner1.coordinates(), corner2.coordinates(), 
                                            corner3.coordinates(), corner4.coordinates()])
    
    elif neighborhood_type=='9': 
        # 创建线段并计算边的中心点
        edge_center1 = ee.Geometry.LineString([corner1.coordinates(), corner2.coordinates()]).centroid()
        edge_center2 = ee.Geometry.LineString([corner2.coordinates(), corner3.coordinates()]).centroid()
        edge_center3 = ee.Geometry.LineString([corner3.coordinates(), corner4.coordinates()]).centroid()
        edge_center4 = ee.Geometry.LineString([corner4.coordinates(), corner1.coordinates()]).centroid()

        # 创建包含所有点的MultiPoint几何对象
        region_coord = ee.Geometry.MultiPoint([
                        corner1.coordinates(), corner2.coordinates(), 
                        corner3.coordinates(), corner4.coordinates(),
                        edge_center1.coordinates(), edge_center2.coordinates(), 
                        edge_center3.coordinates(), edge_center4.coordinates(),
                        point.coordinates()])
    else:
        print('neighborhood_type should be "4" or "9"')

    # 使用MultiPoint几何对象作为geometry参数
    result = image.reduceRegion(
        reducer=ee.Reducer.toList(),  # 转换为列表
        geometry=region_coord,
        scale=scale,
        maxPixels=1e9)

    # 在结果中添加点坐标
    result = result.set('point_coordinates', point.coordinates())
    if Pointfilter:
        result = ee.Algorithms.If(ee.List(result.get('angle')).size().gte(int(neighborhood_type)), result, None)
    return result 

def weighted_avg_func(neighbor):
    # 使用ee.Number转换index，因为原始的index是ee.ComputedObject
    neighbors_info = ee.Dictionary(neighbor)

    # 将邻域点信息转换为ee.Array
    lon_coords = ee.Array(neighbors_info.get('longitude'))
    lat_coords = ee.Array(neighbors_info.get('latitude'))
    elevations = ee.Array(neighbors_info.get('elevation'))
    x_ = ee.Array(neighbors_info.get('x'))
    y_ = ee.Array(neighbors_info.get('y'))
    angles = ee.Array(neighbors_info.get('angle'))
    point_coords  = ee.List(neighbors_info.get('point_coordinates'))
    
    # 计算距离和权重
    point1_x = ee.Number(point_coords.get(0))
    point1_y = ee.Number(point_coords.get(1))
    distances = lon_coords.subtract(point1_x).pow(2).add(lat_coords.subtract(point1_y).pow(2)).sqrt()
    weights = distances.pow(-1)
    sum_weights = weights.reduce(ee.Reducer.sum(), [0])

    # 计算加权平均高程
    weighted_elevations = elevations.multiply(weights)
    weighted_avg_elevation = weighted_elevations.reduce(ee.Reducer.sum(), [0]).divide(sum_weights).get([0])

    # 计算平均角度
    weighted_angles = angles.multiply(weights)
    weighted_avg_angles = weighted_angles.reduce(ee.Reducer.sum(), [0]).divide(sum_weights).get([0])
    
    # 计算平均x
    weighted_X = x_.multiply(weights)
    weighted_avg_X = weighted_X.reduce(ee.Reducer.sum(), [0]).divide(sum_weights).get([0])
    
    # 计算平均y
    weighted_Y = y_.multiply(weights)
    weighted_avg_Y = weighted_Y.reduce(ee.Reducer.sum(), [0]).divide(sum_weights).get([0])

    return ee.Dictionary({'elevation': weighted_avg_elevation,'angle':weighted_avg_angles,
                          'x':weighted_avg_X,'y':weighted_avg_Y,'point_coordinates':neighbors_info.get('point_coordinates')})

def avg_func(neighbor):
    neighbors_info = ee.Dictionary(neighbor)
    elevations = ee.Array(neighbors_info.get('elevation'))
    angles = ee.Array(neighbors_info.get('angle'))
    x_ = ee.Array(neighbors_info.get('x'))
    y_ = ee.Array(neighbors_info.get('y'))
    return ee.Dictionary({'elevation':elevations.reduce(ee.Reducer.mean(),[0]).get([0]) ,
                           'angle':angles.reduce(ee.Reducer.mean(),[0]).get([0]),
                           'x':x_.reduce(ee.Reducer.mean(),[0]).get([0]),
                           'y':y_.reduce(ee.Reducer.mean(),[0]).get([0]),
                           'point_coordinates':neighbors_info.get('point_coordinates')})

def Volum9_func(neighbors_info):
    
    # 修改方程函数以匹配新的模型
    def equation(params, x, y):
        a, b, c, d, e, f = params
        return a*x**2 + b*y**2 + c*x*y + d*x + e*y + f

    # 修改残差函数，去除z作为变量
    def residuals(params, x, y, z_true):
        z_pred = equation(params, x, y)
        return z_pred - z_true
    
    # 根据求得的参数和给定点的经纬度计算高程
    def calculate_z(params, x, y):
        # 根据新模型参数更新，只有6个参数
        a, b, c, d, e, f = params
        # 更新方程以反映新的模型形式
        z_pred = a*x**2 + b*y**2 + c*x*y + d*x + e*y + f
        return z_pred
    

    lon = np.array(neighbors_info['longitude'])
    lat = np.array(neighbors_info['latitude'])
    elevation = np.array(neighbors_info['elevation'])
    angles = np.array(neighbors_info['angle'])
    x_ = np.array(neighbors_info['x'])
    y_ = np.array(neighbors_info['y'])
    lon_pre, lat_pre = np.array(neighbors_info['point_coordinates'])

    # 初始参数猜测
    initial_guess = np.zeros(6)

    # 使用最小二乘法求得的参数,预测高程
    result0 = least_squares(residuals, initial_guess, args=(lon, lat, elevation))
    params0 = result0.x  
    z_pred = calculate_z(params0, lon_pre, lat_pre)

    # 使用最小二乘法求得的参数,预测angle
    result1 = least_squares(residuals, initial_guess, args=(lon, lat, angles))
    params1 = result1.x  
    angle_pred = calculate_z(params1, lon_pre, lat_pre)
    
    # 使用最小二乘法求得的参数,预测x
    result2 = least_squares(residuals, initial_guess, args=(lon, lat, x_))
    params2 = result2.x  
    x_pred = calculate_z(params2, lon_pre, lat_pre)
    
    # 使用最小二乘法求得的参数,预测y
    result3 = least_squares(residuals, initial_guess, args=(lon, lat, y_))
    params3 = result3.x  
    y_pred = calculate_z(params3, lon_pre, lat_pre)
    
    return {'elevation':z_pred,'angle':angle_pred,
            'x':x_pred,'y':y_pred,
            'point_coordinates':[lon_pre, lat_pre]}

def Flat4_func(neighbors_info):

    lon = neighbors_info['longitude']
    lat = neighbors_info['latitude']
    elevation = neighbors_info['elevation']
    angles = np.array(neighbors_info['angle'])
    x_ = np.array(neighbors_info['x'])
    y_ = np.array(neighbors_info['y'])
    lon_pred, lat_pred = np.array(neighbors_info['point_coordinates'])
    
    A = np.vstack([lon, lat, np.ones(len(lon))]).T  # 构建矩阵A

    b = elevation  # 高程向量b
    # 使用最小二乘法解方程Ax = b
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    # x包含拟合参数a, b, c（注意这里不再是1/c）
    a, b, c = x
    z_pred = a * lon_pred + b * lat_pred + c

    b = angles
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    a, b, c = x
    angle_pred = a * lon_pred + b * lat_pred + c
    
    b = x_
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    a, b, c = x
    x_pred = a * lon_pred + b * lat_pred + c
    
    b = y_
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    a, b, c = x
    y_pred = a * lon_pred + b * lat_pred + c

    return {'elevation':z_pred,'angle':angle_pred,'x':x_pred,'y':y_pred,'point_coordinates':[lon_pred, lat_pred]}

def apply_map_to_list(input_list, func): return input_list.map(func)

def apply_map_to_list_local(input_list, func): return map(func,input_list)

Len_Templist_A =  Templist_A.size().getInfo()
pbar = tqdm(range(Len_Templist_A),ncols=80)
Neighbors = '4' # '9'

# 需要注意'Area_elavation','Volum_elavation'涉及数学拟合，GEE-->Python环境
Elvevation_model = 'weighted_avg_elevation' #'weighted_avg_elevation' , 'avg_elevation', 'Area_elavation','Volum_elavation'

All_PointLine = [] 
for i in pbar:
    points = Line2Points(Templist_A.get(i),region=AOI,scale=Prj_scale)  
    points = points.geometry().coordinates()
    All_PointLine.append(points)
    
EE_PointLine = ee.List(All_PointLine)

# 过滤点数不符合要求的辅助线
EE_PointLine = EE_PointLine.map(partial(filter_Listlen,len=3)).removeAll([None])

cal_neighbors = apply_map_to_list(EE_PointLine, 
                                  lambda x: ee.List(x).map(partial(get_neighborhood_info,image=Cal_image,
                                                                scale=Prj_scale,neighborhood_type=Neighbors,
                                                                Pointfilter=True)).removeAll([None]))

cal_neighbors = cal_neighbors.map(partial(filter_Listlen,len=3)).removeAll([None])

if Elvevation_model == 'weighted_avg_elevation':
    Points_WithH_Angle = apply_map_to_list(
                                        cal_neighbors,      
                                        lambda x: ee.List(x).map(weighted_avg_func))
    
elif Elvevation_model == 'avg_elevation':
    Points_WithH_Angle = apply_map_to_list(
                                        cal_neighbors,
                                        lambda x: ee.List(x).map(avg_func))
    
elif (Elvevation_model == 'Area_elavation') or (Elvevation_model == 'Volum_elavation'):

    neighbors_info = cal_neighbors.getInfo() # 非常大
    if Elvevation_model == 'Area_elavation':
        Points_WithH_Angle = list(apply_map_to_list_local(neighbors_info,lambda x: list(map(Flat4_func,x))))
    elif Elvevation_model == 'Volum_elavation':
        Points_WithH_Angle = list(apply_map_to_list_local(neighbors_info,lambda x: list(map(Volum9_func,x))))

100%|██████████████████████████████████████████| 99/99 [00:00<00:00, 196.04it/s]


In [6]:
import json
# Info = Points_WithH_Angle.getInfo()
# with open('Points_WithH_Angle.json', 'w') as json_file:
#     json.dump(Info, json_file, indent=4)
    
with open('Points_WithH_Angle.json', 'r') as file:
    # 读取并解析JSON数据
    data = json.load(file)
    
Points_WithH_Angle = ee.List(data[50:60])

FileNotFoundError: [Errno 2] No such file or directory: 'Points_WithH_Angle.json'

## 计算n阶导数、曲率-->构建局部极大值索引-->根据n阶导数计算局部极大值点、曲率计算凸性点-->根据波峰波谷过滤局部极大值点

In [7]:
# 定义一个函数来计算一阶导数，使导数的数量与原数据相同
def compute_derivative_same_length(elev_list):
    def diff_helper(index):
        index = ee.Number(index)  # 包装 index 为 ee.Number
        current = ee.Number(elev_list.get(index))
        next_ = ee.Number(elev_list.get(index.add(1)))
        prev = ee.Number(elev_list.get(index.subtract(1)))
        return ((next_.subtract(current))
                .add(current.subtract(prev))).divide(2)
    
    # 对 elevation 列表进行微分，得到一阶导数
    first_derivative = ee.List.sequence(1, elev_list.size().subtract(2)).map(diff_helper)
    
    # 使用前向差分和后向差分来计算序列的第一个和最后一个点的导数
    first_point_derivative = ee.Number(elev_list.get(1)).subtract(ee.Number(elev_list.get(0)))
    last_point_derivative = ee.Number(elev_list.get(-1)).subtract(ee.Number(elev_list.get(-2)))
    
    # 将导数序列的第一个和最后一个点的导数添加到导数序列中
    derivative_same_length = ee.List([first_point_derivative]).cat(first_derivative).add(last_point_derivative)
    
    return derivative_same_length

# 计算曲率
def compute_curvature(first_deriv, second_deriv):
    def curvature_helper(index):
        y_prime = ee.Number(first_deriv.get(index))
        y_double_prime = ee.Number(second_deriv.get(index))
        return ee.Number.abs(y_double_prime).divide(y_prime.pow(2).add(1).pow(1.5))
    return ee.List.sequence(0, first_deriv.size().subtract(1)).map(curvature_helper)

# 计算曲率局部极大值的索引
def find_local_maxima_indices(curvature_list):
    # 创建一个包含索引和值的新列表
    indexed_list = curvature_list.zip(ee.List.sequence(0, curvature_list.size().subtract(1)))
    
    def local_maxima_helper(element):
        element = ee.List(element)
        current = ee.Number(element.get(0))
        index = ee.Number(element.get(1))
        prev = ee.Number(curvature_list.get(index.subtract(1)))
        next_ = ee.Number(curvature_list.get(index.add(1)))
        
        # 处理边界情况
        prev = ee.Algorithms.If(index.eq(0), current, prev)
        next_ = ee.Algorithms.If(index.eq(curvature_list.size().subtract(1)), current, next_)
        
        is_local_maxima = current.gt(prev).And(current.gt(next_))
        return ee.Algorithms.If(is_local_maxima, index, None)
    
    maxima_indices = indexed_list.map(local_maxima_helper).removeAll([None])
    return maxima_indices

# 计算曲率局部极小值的索引
def find_local_minima_indices(curvature_list):
    # 创建一个包含索引和值的新列表
    indexed_list = curvature_list.zip(ee.List.sequence(0, curvature_list.size().subtract(1)))
    
    def local_minima_helper(element):
        element = ee.List(element)
        current = ee.Number(element.get(0))
        index = ee.Number(element.get(1))
        prev = ee.Number(curvature_list.get(index.subtract(1)))
        next_ = ee.Number(curvature_list.get(index.add(1)))
        
        # 处理边界情况
        prev = ee.Algorithms.If(index.eq(0), current, prev)
        next_ = ee.Algorithms.If(index.eq(curvature_list.size().subtract(1)), current, next_)
        
        is_local_minima = current.lt(prev).And(current.lt(next_))
        return ee.Algorithms.If(is_local_minima, index, None)
    
    minima_indices = indexed_list.map(local_minima_helper).removeAll([None])
    return minima_indices

# 1、2、3阶导数共同判断
def filter_indices(combined_list):
    def filter_helper(element):
        index = combined_list.indexOf(element)
        derivs = ee.List(element)
        condition = ee.Number(derivs.get(0)).gte(0) \
                    .And(ee.Number(derivs.get(1)).lte(0)) \
                    .And(ee.Number(derivs.get(2)).gte(0))
        return ee.Algorithms.If(condition, index, None)
    indices = combined_list.map(filter_helper)
    # 移除None值
    filtered_indices = indices.removeAll([None])
    return filtered_indices

# 过滤掉微小起伏波动产生的局部极大值点
def filter_local_maxima_by_elevation_difference(local_maxima_indices, local_minima_indices, elevation_list,elevation_difference_threshold=15):
    def filter_helper(maxima_index):
        maxima_index = ee.Number(maxima_index)
        maxima_elevation = ee.Number(elevation_list.get(maxima_index))
        
        # 找到最近的前一个局部极小值点
        minima_before = local_minima_indices.filter(ee.Filter.lt('item', maxima_index))
        nearest_minima_index_before = ee.Algorithms.If(
            minima_before.size().gt(0),
            ee.Number(minima_before.reduce(ee.Reducer.max())),
            -1  # 如果 minima_before 为空，则返回 -1
        )
        # 找到最近的后一个局部极小值点
        minima_after = local_minima_indices.filter(ee.Filter.gt('item', maxima_index))
        nearest_minima_index_after = ee.Algorithms.If(
            minima_after.size().gt(0),
            ee.Number(minima_after.reduce(ee.Reducer.min())),
            -1  # 如果 minima_after 为空，则返回 -1
        )
        
        # 如果两个局部极小值点都存在，则计算高程差异并判断
        condition = ee.Algorithms.If(
            ee.Number(nearest_minima_index_before).gt(-1).And(ee.Number(nearest_minima_index_after).gt(-1)),
            ee.Number(elevation_list.get(nearest_minima_index_before)).subtract(maxima_elevation).abs().gte(elevation_difference_threshold).Or(
                ee.Number(elevation_list.get(nearest_minima_index_after)).subtract(maxima_elevation).abs().gte(elevation_difference_threshold)
            ),
            True  # 如果无法同时获取两端的局部极小值点，则保留该点
        )
        
        return ee.Algorithms.If(condition, maxima_index, None)
    
    filtered_maxima_indices = local_maxima_indices.map(filter_helper).removeAll([None])
    return filtered_maxima_indices

def calculate_extreme_points(elevations,elevation_difference_threshold = 15):
    
    # 计算n阶导数\曲率
    elevations = ee.List(elevations)
    first_derivative = compute_derivative_same_length(elevations)
    second_derivative = compute_derivative_same_length(first_derivative)
    third_derivative = compute_derivative_same_length(second_derivative)
    curvature = compute_curvature(first_derivative, second_derivative)
    # 将三个导数列表组合起来
    combined_derivatives = first_derivative\
                            .zip(second_derivative)\
                            .zip(third_derivative)\
                            .map(lambda x: ee.List(x).flatten())
                            
    filtered_derivatives_indices = filter_indices(combined_derivatives)
    curvature_local_maxima_indices = find_local_maxima_indices(curvature)

    # 计算局部极大值和局部极小值的索引
    elevations_local_maxima_indices = find_local_maxima_indices(elevations)
    elevations_local_minima_indices = find_local_minima_indices(elevations)

    # 过滤掉微小起伏波动产生的局部极大值点
    filtered_elevations_maxima_indices = filter_local_maxima_by_elevation_difference(elevations_local_maxima_indices, 
                                                                        elevations_local_minima_indices, elevations,elevation_difference_threshold)

    # 求n阶导数与曲率交集(凸性点)
    intersection_indices = filtered_derivatives_indices.filter(ee.Filter.inList('item', curvature_local_maxima_indices))

    # 求局部极大值点与凸性点并集
    unionFilter_indices = ee.List(intersection_indices.cat(filtered_elevations_maxima_indices)).distinct().sort()
    return unionFilter_indices

elevations = apply_map_to_list(
    Points_WithH_Angle,
    lambda x: ee.List(x).map(lambda x: ee.Dictionary(x).get('elevation')))

unionFilter_indices = elevations.map(partial(calculate_extreme_points,elevation_difference_threshold = 15))

In [8]:
unionFilter_indices

In [9]:

# 计算n阶导数\曲率
elevations = ee.List([4282.60811986, 4283.92362144, 4279.34871834, 4280.27762334,
       4284.95306453, 4302.37100147, 4308.32150488, 4316.20656617,
       4322.3280081 , 4335.92656043, 4344.7122046 , 4346.95414791,
       4349.61427116, 4353.77746885, 4377.10629843, 4382.31050162,
       4388.15114824, 4388.86604944, 4401.02745263, 4401.40731129,
       4404.65720247, 4415.69268852, 4433.41117708, 4444.83708174,
       4453.04485864, 4461.26023645, 4478.7409936 , 4487.70388003,
       4497.14632655, 4506.94731667, 4520.73085907, 4523.07779733,
       4523.33119259, 4522.94562998, 4524.04548949, 4536.52543639,
       4539.30471801, 4546.63675475, 4549.59182692, 4548.07598295,
       4546.06830433, 4534.03963773, 4532.50399114, 4536.40617711,
       4537.87563049, 4545.12491035, 4558.89261232, 4576.60650374,
       4590.07978563, 4598.35029026, 4601.00486619, 4601.52401497,
       4582.41923291, 4567.03686587, 4550.43976088, 4545.12963889,
       4547.7205447 , 4559.00474891, 4574.38529342, 4593.22249281,
       4613.58797415, 4617.83663513, 4615.54885797, 4608.37599688,
       4607.43444147, 4599.97670371, 4589.46140048, 4579.86386787])
elevation_difference_threshold = 15

first_derivative = compute_derivative_same_length(elevations)
second_derivative = compute_derivative_same_length(first_derivative)
third_derivative = compute_derivative_same_length(second_derivative)
curvature = compute_curvature(first_derivative, second_derivative)
                        
# 计算局部极大值和局部极小值的索引
elevations_local_maxima_indices = find_local_maxima_indices(elevations)
elevations_local_minima_indices = find_local_minima_indices(elevations)
curvature_local_maxima_indices = find_local_maxima_indices(curvature)

# 过滤掉微小起伏波动产生的局部极大值点
filtered_elevations_maxima_indices = filter_local_maxima_by_elevation_difference(elevations_local_maxima_indices, 
                                                                    elevations_local_minima_indices, elevations,elevation_difference_threshold)

# 将三个导数列表组合起来
combined_derivatives = first_derivative\
                        .zip(second_derivative)\
                        .zip(third_derivative)\
                        .map(lambda x: ee.List(x).flatten())
filtered_derivatives_indices = filter_indices(combined_derivatives)

# 求n阶导数与曲率交集(凸性点)
intersection_indices = filtered_derivatives_indices.filter(ee.Filter.inList('item', curvature_local_maxima_indices))

# 求局部极大值点与凸性点并集
unionFilter_indices = ee.List(intersection_indices.cat(filtered_elevations_maxima_indices)).distinct().sort()

## 根据已有选点计算Leftlayover、RigthLayover、Shadow

1、根据union_indices 在 Points_WithH_Angle上确定极值点
2、根据极值点与其余点交互计算Leftlayover、RigthLayover、Shadow（注意升降轨判定方位）
3、将计算点转为栅格，并导出
4、整合代码，形成完整实例

In [32]:
# 定义一个函数来计算两点之间的距离
def calculate_distance(point1, point2,scale=10):
    return ee.Number(point1.get('x')).subtract(point2.get('x')).multiply(ee.Number(scale)).pow(2) \
        .add(ee.Number(point1.get('y')).subtract(point2.get('y')).multiply(ee.Number(scale)).pow(2)).sqrt()

def calculate_Left_Layover(previous_points_left,extreme_point, scale):
     # 计算 elevation 差值、距离差值\计算角度和 Distortion\判断是否存在几何畸变
    differences_Left = previous_points_left.map(lambda point:
        ee.Dictionary({
            'elevation_difference': ee.Number(extreme_point.get('elevation')).subtract(ee.Number(ee.Dictionary(point).get('elevation'))),
            'distance': calculate_distance(extreme_point, ee.Dictionary(point), scale=scale),
            'point_coordinates': ee.Dictionary(point).get('point_coordinates'),
            'Distortion Type': 'Leftlayover',
            'values':1}))

    differences_and_angle_Left = differences_Left.map(lambda item:
        ee.Dictionary(item).set('arcAngle', ee.Algorithms.If(
            ee.Number(ee.Dictionary(item).get('elevation_difference')).gt(0),  # 检查高差是否为正
            ee.Number(ee.Dictionary(item).get('distance')).atan2(ee.Dictionary(item).get('elevation_difference')).multiply(180).divide(ee.Number(math.pi)),  # 计算角度（度）
            -1  # 如果高差为负，不计算角度
        )))

    differences_and_angle_Left = differences_and_angle_Left.map(lambda item:
            ee.Dictionary(item).set(
                    'Distortion', ee.Algorithms.If(
                    ee.Number(ee.Dictionary(item).get('arcAngle')).gt(extreme_point.get('angle')),
                    1, 0)))

    # 创建 Left Features
    features_Left = differences_and_angle_Left.map(lambda item:
        ee.Feature(ee.Geometry.Point(ee.List(ee.Dictionary(item).get('point_coordinates'))),item))
    
    return features_Left

def calculate_Right_Layover(distorted_features_Left,extreme_point, afters_points_Right, scale):
    # 按距离降序排序并获取距离最远的特征
    max_distance_feature = distorted_features_Left.sort('distance', False).first()

    # 获取最远特征的高程差
    max_distance_elevation = ee.Number(max_distance_feature.get('elevation_difference'))

    # 创建一个新的字典并替换 'elevation'
    new_extreme_point = extreme_point.set('elevation', ee.Number(extreme_point.get('elevation')).subtract(max_distance_elevation))

    differences_Right = afters_points_Right.map(lambda point:
        ee.Dictionary({
            'elevation_difference': ee.Number(ee.Dictionary(point).get('elevation')).subtract(ee.Number(new_extreme_point.get('elevation'))),
            'elevation_difference2':ee.Number(extreme_point.get('elevation')).subtract(ee.Number(ee.Dictionary(point).get('elevation'))),
            'distance': calculate_distance(new_extreme_point, ee.Dictionary(point),scale=scale),
            'point_coordinates':ee.Dictionary(point).get('point_coordinates')}))

    differences_and_angle_Right = differences_Right.map(lambda item:
        ee.Dictionary(item).set('arcAngle', ee.Algorithms.If(
            ee.Number(ee.Dictionary(item).get('elevation_difference')).gt(0)
            .And(ee.Number(ee.Dictionary(item).get('elevation_difference2')).gt(0)),
            ee.Number(ee.Dictionary(item).get('distance')).atan2(ee.Dictionary(item).get('elevation_difference')).multiply(180).divide(ee.Number(math.pi)),  # 计算角度（度）
            -1  # 如果任一高差为负，不计算角度
        )))

    differences_and_angle_Right = differences_and_angle_Right.map(lambda item:
            ee.Dictionary(item).set(
                'Distortion', ee.Algorithms.If(
                    ee.Number(ee.Dictionary(item).get('arcAngle')).gt(extreme_point.get('angle')),
                    1,0)))
    
    features_Right = differences_and_angle_Right.map(lambda item:
                                    ee.Feature(
                                        ee.Geometry.Point(ee.List(ee.Dictionary(item).get('point_coordinates'))),
                                        {'Distortion': ee.Dictionary(item).get('Distortion'),
                                        'Distortion Type':'Rightlayover',
                                        'values':3}
                                    ))
    
    # 筛选出存在几何畸变的特征
    distorted_features_Right = ee.FeatureCollection(features_Right).filter(ee.Filter.eq('Distortion', 1))

    # 判断是否存在几何畸变的特征,将顶点追加进去
    has_distortion = ee.Algorithms.If(
        distorted_features_Right.size().gt(0),
        features_Right.add(ee.Feature(
            ee.Geometry.Point(ee.List(extreme_point.get('point_coordinates'))),
            {'Distortion': 1}
        )),
        features_Right
    )

    # 将结果转换为列表
    features_Right = ee.List(has_distortion)
    return features_Right

def calculate_Shadow(afters_points_Right,extreme_point,scale):

    differences_Right = afters_points_Right.map(lambda point:
        ee.Dictionary({
            'elevation_difference': ee.Number(extreme_point.get('elevation')).subtract(ee.Number(ee.Dictionary(point).get('elevation'))),
            'distance': calculate_distance(extreme_point, ee.Dictionary(point),scale=scale),
            'point_coordinates':ee.Dictionary(point).get('point_coordinates')}))

    differences_and_angle_Right = differences_Right.map(lambda item:
        ee.Dictionary(item).set('arcAngle', ee.Algorithms.If(
            ee.Number(ee.Dictionary(item).get('elevation_difference')).gt(0),  # 检查高差是否为正
            ee.Number(ee.Dictionary(item).get('distance')).atan2(ee.Dictionary(item).get('elevation_difference')).multiply(180).divide(ee.Number(math.pi)),  # 计算角度（度）
            -1  # 如果高差为负，不计算角度
        )))

    differences_and_angle_Right = differences_and_angle_Right.map(lambda item:
            ee.Dictionary(item).set(
                'Distortion', ee.Algorithms.If(
                    ee.Number(ee.Dictionary(item).get('arcAngle')).gt(ee.Number(90).subtract(extreme_point.get('angle'))),
                    1,0)))
    
    Shadow = differences_and_angle_Right.map(lambda item:
                                    ee.Feature(
                                        ee.Geometry.Point(ee.List(ee.Dictionary(item).get('point_coordinates'))),
                                        {'Distortion': ee.Dictionary(item).get('Distortion'),
                                        'Distortion Type':'Shadow',
                                        'values':5}))
    return Shadow

def calculate_Distor(index, pointList, Candidate_Distor_Points=10, scale=10):

    # 获取极值点
    extreme_point = ee.Dictionary(pointList.get(index))

    # 获取极值点前十位数的索引范围及前十位数的点
    start_index_left = ee.Number(index).subtract(Candidate_Distor_Points).max(0)  # 确保索引不小于 0
    end_index_left = index
    previous_points_left = pointList.slice(start_index_left, end_index_left)
    
    start_index_Right = ee.Number(index).add(Candidate_Distor_Points).min(pointList.size().subtract(1))  # 确保索引不大于 pointList 的最大索引
    end_index_Right = index
    afters_points_Right = pointList.slice(ee.Number(end_index_Right).add(1),start_index_Right.add(1)).reverse()

    # ----------------------------------------Left layover----------------------------------------
    features_Left = calculate_Left_Layover(previous_points_left,extreme_point, scale)

    # ----------------------------------------Right layover----------------------------------------
    ## 当存在Left layover才执行Right layover判断
    distorted_features_Left = ee.FeatureCollection(features_Left).filter(ee.Filter.eq('Distortion', 1))
    features_Right = ee.Algorithms.If(
        distorted_features_Left.size().gt(0),
        calculate_Right_Layover(distorted_features_Left,extreme_point, afters_points_Right, scale),
        ee.List([]))

    # ----------------------------------------Shadow----------------------------------------
    Shadow = calculate_Shadow(afters_points_Right,extreme_point,scale)
    
    return ee.List([features_Left,features_Right,Shadow])

def Distor_line(pointList,Filter_indices,Candidate_Distor_Points = 10,scale=10):
    
    # 使用 unionFilter_indices 映射函数
    Distor_feature = Filter_indices.map(lambda index: calculate_Distor(
                                                ee.Number(index), pointList=pointList, 
                                                Candidate_Distor_Points=Candidate_Distor_Points, 
                                                scale=scale)).flatten()
    return Distor_feature

def process_pair(pair,Candidate_Distor_Points = 10,scale=10):
    pair_list = ee.List(pair)
    pointList = ee.List(pair_list.get(0))
    Filter_indices = ee.List(pair_list.get(1))
    return Distor_line(pointList, Filter_indices,Candidate_Distor_Points = Candidate_Distor_Points,scale=scale)

paired_lists = Points_WithH_Angle.zip(unionFilter_indices)

# 对每对列表应用 Distor_line 函数
Distor_FeatureList = paired_lists.map(partial(process_pair,Candidate_Distor_Points=10,scale=Origin_scale))
Distor_featureCollection = ee.FeatureCollection(Distor_FeatureList.flatten())

Leftlayover  = Distor_featureCollection.filter(ee.Filter.And(
                    ee.Filter.eq('Distortion', 1),
                    ee.Filter.eq('Distortion Type', 'Leftlayover')))
Rightlayover = Distor_featureCollection.filter(ee.Filter.And(
                    ee.Filter.eq('Distortion', 1),
                    ee.Filter.eq('Distortion Type', 'Rightlayover')))
Shadow       = Distor_featureCollection.filter(ee.Filter.And(
                    ee.Filter.eq('Distortion', 1),
                    ee.Filter.eq('Distortion Type', 'Shadow')))

In [33]:
# Distor_Image = Cal_image.select('Leftlayover').paint(Leftlayover, 'values')\
#     .addBands(Cal_image.select('Rightlayover').paint(Rightlayover, 'values'))\
#     .addBands(Cal_image.select('Shadow').paint(Shadow, 'values')).reproject(crs=Projection, scale=Prj_scale).clip(AOI)

LeftLayover = ee.Image().paint(Leftlayover, 'values').reproject(crs=Projection, scale=Prj_scale).clip(AOI).toInt8()
DataIO.Geemap_export('LeftLayover.tif',LeftLayover,region=AOI,scale=Prj_scale,rename_image=False)

Rightlayover = ee.Image().paint(Rightlayover, 'values').reproject(crs=Projection, scale=Prj_scale).clip(AOI).toInt8()
DataIO.Geemap_export('Rightlayover.tif',Rightlayover,region=AOI,scale=Prj_scale,rename_image=False)

Shadow = ee.Image().paint(Shadow, 'values').reproject(crs=Projection, scale=Prj_scale).clip(AOI).toInt8()
DataIO.Geemap_export('Shadow.tif',Shadow,region=AOI,scale=Prj_scale,rename_image=False)

Generating URL ...


In [ ]:
# 对比计算结果，验证准确性